<a href="https://colab.research.google.com/github/UznetDev/M5-H2-Regression-Competition/blob/main/07_Noy_2024_home_work_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import & install

In [11]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.6 MB/s eta 0:00:00


In [12]:
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt

## Load `train.csv`

In [5]:
df = pd.read_csv('train.csv')
df.head(1)

,id,Row#,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds,yield
0,0,104.0,12.5,0.25,0.25,0.25,0.75,69.7,42.1,58.2,50.2,24.3,41.2,16.0,0.26,0.477941,0.423927,34.043022,6079.08526


## Functions

In [3]:
def print_metrics(model, X_train, X_test, y_train, y_test, cv_mae):
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    r2 = r2_score(y_test, y_test_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

    text = f"""
    Model Performance Metrics:
    --------------------------
    CV Mean MAE: {-cv_mae.mean():.2f}, {-cv_mae.std():.2f}

           Train  | Test
         |-----------------|
     MAE | {train_mae:.2f} | {test_mae:.2f} |
         |-----------------|
    RMSE | {train_rmse:.2f} | {test_rmse:.2f} |
         |-----------------|

        R^2 Score (Test): {r2:.2f}
    """
    print(text)

## Split data

In [6]:
X = df.drop('yield', axis=1)
y = df['yield']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Models

In [7]:
rf_model = RandomForestRegressor(max_depth=9, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=9, random_state=42)

In [8]:
dt_model = DecisionTreeRegressor(max_depth=9, random_state=42)
dt_model.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=9, random_state=42)

In [9]:
ln_model = LinearRegression()
ln_model.fit(X_train, y_train)

LinearRegression()

## Get weight model with `optune`

In [23]:
def objective(trial):
    rf_weight = trial.suggest_float("rf_weight", -1, 1)
    dt_weight = trial.suggest_float("dt_weight", -1, 1)
    ln_weight = trial.suggest_float("ln_weight", -1, 1)

    total_weight = rf_weight + dt_weight + ln_weight
    rf_weight /= total_weight
    dt_weight /= total_weight
    ln_weight /= total_weight

    rf_pred = rf_model.predict(X_test)
    dt_pred = dt_model.predict(X_test)
    ln_pred = ln_model.predict(X_test)

    blended_pred = (rf_weight * rf_pred) + (dt_weight * dt_pred) + (ln_weight * ln_pred)

    mse = mean_absolute_error(y_test, blended_pred)
    return mse

In [24]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1000)

[I 2024-11-11 14:16:53,819] A new study created in memory with name: no-name-b5a10e7f-60e8-4181-a1fb-6b52103fc084
[I 2024-11-11 14:16:53,862] Trial 0 finished with value: 264.8207752971018 and parameters: {'rf_weight': -0.906403113718663, 'dt_weight': 0.4657710878435508, 'ln_weight': -0.2246000099079859}. Best is trial 0 with value: 264.8207752971018.
[I 2024-11-11 14:16:53,933] Trial 1 finished with value: 247.45760239004935 and parameters: {'rf_weight': 0.9250190469010395, 'dt_weight': -0.22238985451384674, 'ln_weight': 0.13543371794410386}. Best is trial 1 with value: 247.45760239004935.
[I 2024-11-11 14:16:53,988] Trial 2 finished with value: 434.88431545842064 and parameters: {'rf_weight': 0.5795671377305305, 'dt_weight': 0.3640864115022935, 'ln_weight': -0.6511496198966524}. Best is trial 1 with value: 247.45760239004935.
[I 2024-11-11 14:16:54,043] Trial 3 finished with value: 264.04956871550877 and parameters: {'rf_weight': 0.16452472568049092, 'dt_weight': 0.5243097000955224, 

In [25]:
print("Optimal weights:")
print("RandomForest weight:", study.best_params["rf_weight"])
print("DecisionTree weight:", study.best_params["dt_weight"])
print("LinearRegression weight:", study.best_params["ln_weight"])

Optimal weights:
RandomForest weight: 0.6352449313835341
DecisionTree weight: -0.030054976436792558
LinearRegression weight: 0.03396478296137406


In [26]:
rf_weight = study.best_params["rf_weight"]
best_dt_weight = study.best_params["dt_weight"]
best_ln_weight = study.best_params["ln_weight"]
total_weight = best_rf_weight + best_dt_weight + best_ln_weight
best_rf_weight /= total_weight
best_dt_weight /= total_weight
best_ln_weight /= total_weight

def blended_predict(X):
    rf_pred = rf_model.predict(X)
    dt_pred = dt_model.predict(X)
    ln_pred = ln_model.predict(X)
    return best_rf_weight * rf_pred + best_dt_weight * dt_pred + best_ln_weight * ln_pred

In [28]:
final_predictions = blended_predict(X_test)
mae = mean_absolute_error(y_test, final_predictions)
r2 = r2_score(y_test, final_predictions)
rmse = np.sqrt(mean_squared_error(y_test, final_predictions))

print(f"MAE: {mae:.2f}")
print(f"R^2 Score: {r2:.2f}")
print(f"RMSE: {rmse:.2f}")

MAE: 244.63
R^2 Score: 0.93
RMSE: 370.73
